In [68]:
names=["Hamilton","Verstappen","Russel","Leclerc","Perez","Sainz","Norris","Ricciardo","Gasly","Alanso","Ocon","Hulkenberg","Stroll","Bottas","Tsunoda","Guanyu","Albon","Latifi","Schumacker","Magnussen"]
price=[31,30.5,24,18,17.5,17,16,14.5,13.5,12.5,12,11.5,9.5,9,8.5,8.0,7.5,7.0,6.5,5.5]
points=[387.5,395.5,16,159,190,164.5,160,115,110,81,74,0,34,226,32,0,0,7,0,0]

In [71]:
from ortools.sat.python import cp_model
# DATA
capacity = 6
budget = 100 #mill



drivers_constructors = [ 
  {"name":"Hamilton",     "type":"driver","price":31, "score":387.5},
  {"name":"Verstappen",   "type":"driver","price":30.5, "score":395.5},
  {"name":"Russel",       "type":"driver","price":24, "score":226}, #Changed
  {"name":"Leclerc",      "type":"driver","price":18, "score":159},
  {"name":"Perez",        "type":"driver","price":17.5, "score":190},
  {"name":"Sainz",        "type":"driver","price":17, "score":164.5},
  {"name":"Norris",       "type":"driver","price":16, "score":160},
  {"name":"Ricciardo",    "type":"driver","price":14.5, "score":115},
  {"name":"Gasly",        "type":"driver","price":13.5, "score":110},
  {"name":"Alanso",       "type":"driver","price":12.5, "score":81},
  {"name":"Ocon",         "type":"driver","price":12, "score":74},
  {"name":"Hulkenberg",   "type":"driver","price":11.5, "score":43}, #Changed
  {"name":"Stroll",       "type":"driver","price":9.5, "score":34},
  {"name":"Bottas",       "type":"driver","price":9, "score":10},    #Changed
  {"name":"Tsunoda",      "type":"driver","price":8.5, "score":32},
  {"name":"Guanyu",       "type":"driver","price":8.0, "score":3},   #Changed
  {"name":"Albon",        "type":"driver","price":7.5, "score":16},  #Changed
  {"name":"Latifi",       "type":"driver","price":7.0, "score":7},
  {"name":"Schumacker",   "type":"driver","price":6.5, "score":0},
  {"name":"Magnussen",    "type":"driver","price":5.5, "score":1},   #Changed

  {"name":"MERCEDES",                      "type":"constructors","price":34.5, "score":613.5},
  {"name":"RED BULL RACING HONDA",         "type":"constructors","price":32.5, "score":585.5},
  {"name":"FERRARI",                       "type":"constructors","price":25, "score":323.5}, 
  {"name":"MCLAREN MERCEDES",              "type":"constructors","price":18.5, "score":275},
  {"name":"ALPINE RENAULT",                "type":"constructors","price":14, "score":155},
  {"name":"ALPHATAURI HONDA",              "type":"constructors","price":10.5, "score":142},
  {"name":"ASTON MARTIN MERCEDES",         "type":"constructors","price":11.5, "score":77}, 
  {"name":"WILLIAMS MERCEDES",             "type":"constructors","price":7, "score":23}, 
  {"name":"ALFA ROMEO RACING FERRARI",     "type":"constructors","price":8, "score":13},
  {"name":"HAAS FERRARI",                  "type":"constructors","price":6, "score":0},]


# PREPROCESSING
n_drivers_constructors = len(set([entry['name'] for entry in drivers_constructors]))

# MODEL
model = cp_model.CpModel()
x_select = [model.NewBoolVar('') for i in range(n_drivers_constructors)]

# Capacity constrain
model.Add(sum(x_select) == capacity)

# Budget constrain -> # ASSUMPTION: * 10 makes all the values integral
model.Add(sum([x_select[i] * int(round(drivers_constructors[i]['price']*10)) for i in range(n_drivers_constructors)]) <= budget * 10)

# Constructor constrain
model.Add(sum([x_select[i] for i in range(n_drivers_constructors) if drivers_constructors[i]['type'] == 'constructors']) == 1)


# Maximize sum of scores selected -> # ASSUMPTION: * 10 makes all the values integral
model.Maximize(sum([x_select[i] * int(round(drivers_constructors[i]['score']*10)) for i in range(n_drivers_constructors)]))

# SOLVE
solver = cp_model.CpSolver()
solver.parameters.log_search_progress = True
model.Proto().objective.scaling_factor = -1./10         #Inverse scaling for solver logging output
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  selected = [i for i in range(n_drivers_constructors) if solver.Value(x_select[i]) == 1]
  print("\n".join([str(drivers_constructors[i]) for i in selected]))


Starting CP-SAT solver v9.3.10497
Parameters: log_search_progress: true
Setting number of workers to 12

Initial optimization model '':
#Variables: 30 (#bools:28 in objective)
  - 30 in [0,1]
#kLinearN: 3 (#terms: 70)

Starting presolve at 0.00s
[ExtractEncodingFromLinear] #potential_supersets=1 #potential_subsets=0 #at_most_one_encodings=0 #exactly_one_encodings=0 #unique_terms=0 #multiple_terms=0 #literals=0 time=3.791e-06s
[Probing] deterministic_time: 1.921e-05 (limit: 1) wall_time: 5.888e-05 (30/30)
[Probing]  - new fixed Boolean: 2 (2/30)
[Probing]  - new binary clause: 2
[SAT presolve] num removable Booleans: 2 / 30
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:36 literals:72 vars:13 one_side_vars:13 simple_definition:0 singleton_clauses:0
[SAT presolve] [2.1529e-05s] clauses:9 literals:18 vars:13 one_side_vars:13 simple_definition:0 singleton_clauses:0
[SAT presolve] [2.7868e-05s] clauses:9 literals:18 vars:13 one_side_vars:13 simple_definition:0 singleton_